# 数据准备

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pymysql
import warnings
import re
warnings.filterwarnings('ignore')

In [2]:
# 导入数据
path = r'L:\项目配仓\西北农林科技大学\二期招培就\数据\招生\本科数据\\'
data_ori1 = pd.read_excel(path+r'2019-2022年录取专业志愿情况数据库.xlsx', sheet_name='2022年')
data_ori2 = pd.read_excel(path+r'2019-2022年分专业招生计划.xlsx', sheet_name='2022')
data_ori3 = pd.read_excel(path+r'2019-2022年录取学生学前信息（周海渊需求）.xlsx', sheet_name='2022')

In [3]:
conn = pymysql.connect(host = 'am-wz9el267w54i2r7ip131930o.ads.aliyuncs.com' # 连接名称，默认127.0.0.1
,user = 'yjy_user' # 用户名
,passwd='Yjy123456' # 密码
,port= 3306 # 端口，默认为3306
,db='standard_code'#数据库名称
,charset='utf8' # 字符编码
)

# 函数定义

In [4]:
# 熵值法
def cal_weight(column,row,value):
    p= np.array([[0.0 for i in range(column)] for i in range(row)])     
    for i in range(column):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
    e=-1/np.log(row)*sum(p*np.log(p+1e-10))      #计算熵值
    g=1-e     # 计算一致性程度
    w=g/sum(g)     #计算权重
    return w

# 志愿填报计算

In [5]:
# 一志愿报考率
data_qu = data_ori1[['录取专业','录取专业志愿顺序号','省份']]
data_group = data_qu[['录取专业','录取专业志愿顺序号']].groupby('录取专业').count().reset_index()
data_group = data_group.rename(columns={'录取专业志愿顺序号':'总人数'})
data_qu['一志愿标记'] = data_qu.apply(lambda x:1 if x['录取专业志愿顺序号']==1 else 0,axis=1)
data_group = data_group.merge(data_qu[['录取专业','一志愿标记']].groupby('录取专业').sum().reset_index(),on='录取专业')
data_group['一志愿报考率'] = data_group['一志愿标记']/data_group['总人数']

In [6]:
# 一二志愿比
data_qu['一二志愿标记'] = data_qu.apply(lambda x:1 if x['录取专业志愿顺序号']==1 or x['录取专业志愿顺序号']==2 else 0,axis=1)
data_group = data_group.merge(data_qu[['录取专业','一二志愿标记']].groupby('录取专业').sum().reset_index(),on='录取专业')
data_group['一二志愿比'] = data_group['一二志愿标记']/data_group['总人数']

In [7]:
# 调剂比例
data_qu['调剂标记'] = data_qu.apply(lambda x:1 if x['录取专业志愿顺序号']=='调剂' else 0,axis=1)
data_group = data_group.merge(data_qu[['录取专业','调剂标记']].groupby('录取专业').sum().reset_index(),on='录取专业')
data_group['调剂比例'] = data_group['调剂标记']/data_group['总人数']

In [8]:
# 本省一志愿
shengfen = '甘肃'
data_qu['本省一志愿标记'] = data_qu.apply(lambda x:1 if x['录取专业志愿顺序号']==1 and x['省份']==shengfen else 0,axis=1)
data_group = data_group.merge(data_qu[['录取专业','本省一志愿标记']].groupby('录取专业').sum().reset_index(),on='录取专业')
data_group['本省一志愿'] = data_group['本省一志愿标记']/data_group['总人数']

In [9]:
w = cal_weight(4,data_group.shape[0],data_group[['一志愿报考率','一二志愿比','调剂比例','本省一志愿']].values)
data_group['志愿填报总分']=data_group.apply(lambda x:x['一志愿报考率']*w[0]+x['一二志愿比']*w[1]+(1-x['调剂比例'])*w[2]+x['本省一志愿']*w[3],axis=1)

# 生源质量

In [10]:
sql = '''SELECT `学校名称` FROM `dim_senior_school`
'''
data_ori4 = pd.read_sql(con=conn,sql=sql)

In [11]:
data_ori4['学校名称'] = data_ori4['学校名称'].str.replace('\n\n','')
data_qu = data_ori3[['录取专业','毕业中学']].merge(pd.DataFrame(data_ori4['学校名称'].str.split('（').explode().str.replace('）','').unique(),columns=['学校名称']),left_on='毕业中学',right_on='学校名称',how='left')
data_qu = data_qu.fillna(0)

In [12]:
data_qu

,录取专业,毕业中学,学校名称
0,环境设计,太原市文华中学校,0
1,环境设计,太原市第三十中学校,0
2,环境设计,太谷二中启航学校,0
3,环境设计,孝义市第三中学校,0
4,动物科学(生物育种),太原师范学院附属中学,太原师范学院附属中学
...,...,...,...
5695,工商管理类,北京师范大学燕化附属中学,0
5696,工商管理类,广东惠州市华罗庚中学,0
5697,园艺(园艺类),山东省济南市长清区第一中学,0
5698,园艺(园艺类),湖北省武汉市汉铁高级中学,0


In [13]:
data_qu['重点中学人数'] = data_qu.apply(lambda x:0 if x['学校名称']==0 else 1,axis=1)
data_group = data_group.merge(data_qu[['录取专业','重点中学人数']].groupby('录取专业').sum().reset_index(),on='录取专业')
data_group['重点中学率'] = data_group['重点中学人数']/data_group['总人数']

# 招生工作完成度计算

In [14]:
data_group = data_ori2.merge(data_group,left_on='专业',right_on='录取专业')

In [15]:
data_group['招生工作完成度']=data_group.apply(lambda x:x['总人数']/x['计划数'] if x['总人数']/x['计划数']<1 else 1,axis=1)

# 算总分

In [16]:
w = cal_weight(3,data_group.shape[0],data_group[['志愿填报总分','重点中学率','招生工作完成度']].values)

In [17]:
data_group['招生总分']=data_group.apply(lambda x:x['志愿填报总分']*w[0]+x['重点中学率']*w[1]+x['招生工作完成度']*w[2],axis=1)
data_group = data_group.rename(columns={'专业':'专业名称'})
data_group[['专业名称','志愿填报总分','重点中学率','招生工作完成度','招生总分']]

,专业名称,志愿填报总分,重点中学率,招生工作完成度,招生总分
0,法学,0.661794,0.216495,1.000000,0.301997
1,社会学类,0.633324,0.208333,1.000000,0.289987
2,公共管理类,0.609651,0.278351,1.000000,0.342125
3,英语,0.619835,0.288889,1.000000,0.352583
4,汉语国际教育,0.655258,0.406250,1.000000,0.454243
5,俄语,0.499903,0.233333,1.000000,0.284863
6,农学(农学类),0.630451,0.236842,1.000000,0.312501
7,智慧农业,0.664217,0.218750,1.000000,0.304281
8,植物保护,0.542479,0.246032,1.000000,0.303222
9,制药工程,0.615557,0.250000,1.000000,0.320315
